# Counter examples

Generates counterexamples for the Taiwan Assignment Mechanism (TAM) and the Chinese Parallel Mechanism (CPM) to show that neither necessarily converges to the student optimal stable matching.

The TAM example is unstable and does not Pareto dominate the SOSM from the student perspective.

The CPM example is unstable but does Pareto dominate the SOSM from the student perspective.

## Preliminaries & functions

In [1]:
import pickle
from tabulate import tabulate
from TaiwanMechanism import *

In [2]:
n_iter = 25

In [3]:
num_to_letter = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', -1: ''}
vectorized_conversion = np.vectorize(num_to_letter.get)

# Label students 0,...,3 as Student A,...,D

In [9]:
def compute_iterations(sim):
    """
    :param sim: TaiwanMechanism instance
    :return: integer representing the iteration at which the simulation stabilizes, numpy array of reports, numpy array of assignments
    """
    stu = sim.students
    sch = sim.schools

    sim.stu_report = sim.stu_pref.copy()
    sosm_indices = np.full(stu, np.nan)
    for j in range(stu):
        sosm_indices[j] = np.where(sim.stu_pref[j] == sim.stu_opt[j])[0][0]

    reports_t = np.full((n_iter, stu, sch), np.nan)
    assignments_t = np.full((n_iter, stu), np.nan)
    indices_t = np.full((n_iter, stu), np.nan)
    for t in range(n_iter):
        if t == 0:
            sim.stu_report = sim.stu_pref.copy()
        reports_t[t] = sim.stu_report.copy()
        assignments_t[t] = sim.assign()
        sim.smart_adjustment()
    for j in range(stu):
        for t in range(n_iter):
            if assignments_t[t, j] == -1:
                indices_t[t, j] = -1
            else:
                indices_t[t, j] = np.where(sim.stu_pref[j] == assignments_t[t, j])[0][0]

    turned_stable = n_iter
    for t in range(1, n_iter):
        if np.all(reports_t[t] == reports_t[t-1]):
            turned_stable = min(turned_stable, t)
        else:
            turned_stable = n_iter
    return turned_stable, reports_t, assignments_t


In [55]:
def print_tables(sim, stu_reports, assign, stable_round):
    stu_labels = np.array([f"Student {vectorized_conversion(i)}" for i in range(sim.students)])
    school_labels = np.array([f"School {i+1}" for i in range(sim.schools)])

    print(f"Student preferences: \n")
    student_prefs = sim.stu_pref + 1
    student_prefs = np.column_stack((stu_labels, student_prefs))
    print(tabulate(student_prefs, tablefmt="latex"))
    print(f'\n')

    print(f"Unadjusted scores: \n")
    scores = np.round(sim.compute_rankings(sim.stu_pref, da=True, return_scores=True), 1)
    scores = np.column_stack((school_labels, scores))
    print(tabulate(scores, tablefmt="latex"))
    print(f'\n')

    print(f'School preferences: \n')
    school_prefs = sim.compute_rankings(stu_reports[0], da=True, return_dict=False)
    school_prefs = vectorized_conversion(school_prefs)
    school_prefs = np.column_stack((school_labels, school_prefs)).astype(str)
    print(tabulate(school_prefs, tablefmt="latex"))
    print(f'\n')

    print(f'SOSM: \n')
    print(sim.stu_opt + 1)
    print(f'\n')

    for curr_round in range(stable_round):
        print(f"Round {curr_round+1}")
        curr_rep = stu_reports[curr_round] + 1
        curr_rep = curr_rep.astype(int).astype(str)
        curr_rep[curr_rep == '0'] = ''
        curr_rep = np.column_stack((stu_labels, curr_rep))
        print(f'Student reports: \n')
        print(tabulate(curr_rep, tablefmt="latex"))
        print(f'\n')

        adj_scores = np.round(sim.compute_rankings(stu_reports[curr_round],return_scores=True), 1).astype(str)
        adj_scores[adj_scores == '-0'] = '0'
        adj_scores[adj_scores == '-990.0'] = ''

        sch_prefs = sim.compute_rankings(stu_reports[curr_round], da=False, return_dict=False)
        for i in range(adj_scores.shape[0]):
            for j in range(adj_scores.shape[1]):
                if adj_scores[i, j] == '':
                    sch_prefs[i][np.where(sch_prefs[i] == j)[0]] = -1
        sch_prefs = vectorized_conversion(sch_prefs)

        sch_prefs = np.column_stack((school_labels, sch_prefs)).astype(str)
        adj_scores = np.column_stack((school_labels, adj_scores))

        print(f'Adjusted scores: \n')
        print(tabulate(adj_scores, tablefmt="latex"))
        print(f'\n')
        print(f'School preferences: \n')
        print(tabulate(sch_prefs, tablefmt="latex"))
        print(f'\n')

        print(f'Assignment: \n')
        print(assign[curr_round] + 1)
        print(f'\n')



## CPM counterexample

In [46]:
schools = 4
students = 4
cap = students // schools
rho = 0.5
deduct = np.array([0.0, 0.0, 3.0, 3.0])

student_pref = np.array([
        [0, 1, 2, 3],
        [0, 1, 3, 2],
        [2, 1, 0, 3],
        [1, 0, 2, 3]
    ])

school_pref_params = np.array([
    [0.9, 1.4, 0.4, 1.8],
    [1.6, 0.9, 0.7, 1.4],
    [0.9, 0.5, 0.4, 1.0],
    [0.9, 0.5, 1.3, 1.0]
])

In [47]:
cpm_sim = TaiwanMechanism(schools, students, rho=rho, deduct=deduct, cap=cap,
                          student_pref=student_pref, student_rep=None,
                          school_pref_type='manual', school_pref_params=school_pref_params,
                          comp_opt=False)

In [48]:
turned_stable_cpm, reports_cpm, assignments_cpm = compute_iterations(cpm_sim)

In [49]:
turned_stable_cpm

2

In [50]:
assert turned_stable_cpm < n_iter

In [52]:
reports_cpm[0:turned_stable_cpm]+1

array([[[1., 2., 3., 4.],
        [1., 2., 4., 3.],
        [3., 2., 1., 4.],
        [2., 1., 3., 4.]],

       [[1., 2., 3., 4.],
        [2., 4., 3., 0.],
        [3., 2., 1., 4.],
        [2., 1., 3., 4.]]])

In [54]:
assignments_cpm[0:turned_stable_cpm]+1

array([[2., 4., 3., 1.],
       [1., 4., 3., 2.]])

### Generate tables

In [56]:
print_tables(
    cpm_sim,
    reports_cpm,
    assignments_cpm,
    turned_stable_cpm
)

Student preferences: 

\begin{tabular}{lrrrr}
\hline
 Student A & 1 & 2 & 3 & 4 \\
 Student B & 1 & 2 & 4 & 3 \\
 Student C & 3 & 2 & 1 & 4 \\
 Student D & 2 & 1 & 3 & 4 \\
\hline
\end{tabular}


Unadjusted scores: 

\begin{tabular}{lrrrr}
\hline
 School 1 & 0.9 & 1.4 & 0.4 & 1.8 \\
 School 2 & 1.6 & 0.9 & 0.7 & 1.4 \\
 School 3 & 0.9 & 0.5 & 0.4 & 1   \\
 School 4 & 0.9 & 0.5 & 1.3 & 1   \\
\hline
\end{tabular}


School preferences: 

\begin{tabular}{lllll}
\hline
 School 1 & D & B & A & C \\
 School 2 & A & D & B & C \\
 School 3 & D & A & B & C \\
 School 4 & C & D & A & B \\
\hline
\end{tabular}


SOSM: 

[2 4 3 1]


Round 1
Student reports: 

\begin{tabular}{lrrrr}
\hline
 Student A & 1 & 2 & 3 & 4 \\
 Student B & 1 & 2 & 4 & 3 \\
 Student C & 3 & 2 & 1 & 4 \\
 Student D & 2 & 1 & 3 & 4 \\
\hline
\end{tabular}


Adjusted scores: 

\begin{tabular}{lrrrr}
\hline
 School 1 &  0.9 &  1.4 & -2.6 &  1.8 \\
 School 2 &  1.6 &  0.9 &  0.7 &  1.4 \\
 School 3 & -2.1 & -2.5 &  0.4 & -2   \\

## TAM counterexample

In [57]:
schools = 4
students = 4
cap = students // schools
rho = 0.5
deduct = np.array([0.0, 0.3, 0.5, 0.8])

student_pref = np.array([
        [2, 0, 1, 3],
        [0, 1, 2, 3],
        [0, 1, 2, 3],
        [1, 3, 0, 2]
    ])

school_pref_params = np.array([
    [0.2, 1.2, 0.7, 1.1],
    [1.2, 1.7, 0.7, 0.5],
    [0.2, 0.8, 1.2, 0.5],
    [0.6, 0.8, 1.1, 0.5]
])

In [58]:
tam_sim = TaiwanMechanism(schools, students, rho=rho, deduct=deduct, cap=cap,
                          student_pref=student_pref, student_rep=None,
                          school_pref_type='manual', school_pref_params=school_pref_params,
                          comp_opt=False)

In [59]:
turned_stable_tam, reports_tam, assignments_tam = compute_iterations(tam_sim)

In [60]:
assert turned_stable_tam < n_iter

In [61]:
turned_stable_cpm

2

### Generate tables

In [62]:
print_tables(
    tam_sim,
    reports_tam,
    assignments_tam,
    turned_stable_tam
)

Student preferences: 

\begin{tabular}{lrrrr}
\hline
 Student A & 3 & 1 & 2 & 4 \\
 Student B & 1 & 2 & 3 & 4 \\
 Student C & 1 & 2 & 3 & 4 \\
 Student D & 2 & 4 & 1 & 3 \\
\hline
\end{tabular}


Unadjusted scores: 

\begin{tabular}{lrrrr}
\hline
 School 1 & 0.2 & 1.2 & 0.7 & 1.1 \\
 School 2 & 1.2 & 1.7 & 0.7 & 0.5 \\
 School 3 & 0.2 & 0.8 & 1.2 & 0.5 \\
 School 4 & 0.6 & 0.8 & 1.1 & 0.5 \\
\hline
\end{tabular}


School preferences: 

\begin{tabular}{lllll}
\hline
 School 1 & B & D & C & A \\
 School 2 & B & A & C & D \\
 School 3 & C & B & D & A \\
 School 4 & C & B & A & D \\
\hline
\end{tabular}


SOSM: 

[3 1 2 4]


Round 1
Student reports: 

\begin{tabular}{lrrrr}
\hline
 Student A & 3 & 1 & 2 & 4 \\
 Student B & 1 & 2 & 3 & 4 \\
 Student C & 1 & 2 & 3 & 4 \\
 Student D & 2 & 4 & 1 & 3 \\
\hline
\end{tabular}


Adjusted scores: 

\begin{tabular}{lrrrr}
\hline
 School 1 & -0.1 & 1.2 & 0.7 &  0.6 \\
 School 2 &  0.7 & 1.4 & 0.4 &  0.5 \\
 School 3 &  0.2 & 0.3 & 0.7 & -0.3 \\
 Scho